# DF Association Rules Group Project - Karl Merisalu and colleagues

<b>Problem:</b> When financial advisors are giving advice they will sometimes recommend clients to purchase specific assets, based on clients' circumstances. To make financial advisors suggestions potentially more relevant for clients, I will create a recommendation engine based on association rules algorithm, which will be recommending asset purchases to clients based on other clients' investments.

Often times, such an analysis is conducted to analyse people's shopping baskets when making purchases in large grocery stores for example. This will help stores to place items that are bought together close to each other for shoppers' convenience (or not, if they want clients to walk through the whole store to pick up their items). We will try to convert this idea to finance sector.

I will be using anonymised client investment data grouped by investment class. In more detail, this shows in which asset classes clients are invested in. By using association rules alorithm, we can hopefully draw conclusions from some clients and make recommendations to others. For example: if many clients are invested in Equities, Multi Assets and Real Estate simultaneously then for a client invested only in Equities and Multi Assets it may be interestig to consider adding Real Estate assets (recommendation) to their portfolio.

### 1) Setting up the enviroment / importing relevant libraries
First we import required libraries which will help us to do data manipulation and run association rules algorithms

In [6]:
import pandas as pd
import numpy as np
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

### 2) Importing the dataset
The data used is based on internal data. We have taken measures to anonymise the data so that no client identifying information is revealed. The data consists of: 1) anonymised client ID and 2) Asset Class the client is invested in

In [7]:
amData = pd.read_csv('instacart\AMdata3.csv')
amData.head()

client id                     AssetClass
0          1                       Equities
1          2                    Multi Asset
2          3  Real Estate & Private Markets
3          4                       Equities
4          5                       Equities

### 3) Getting an overview of fequencies of each asset class
First let's take a look at the distribution of which asset classes do clients invest in

In [8]:
amData['AssetClass'].value_counts().rename("freq")

Equities                         309
Fixed Income                     215
Multi Asset                      148
Real Estate & Private Markets    122
Hedge Funds                       35
Distribution Partners              6
Money Market                       4
Name: freq, dtype: int64

### 4) Grouping data by client id to have an overview of each client id exposure to asset classes
Because some clients have invested in several asset classes, we will group asset classes by client id.

In [9]:
client_baskets = amData.groupby(['client id']).AssetClass.apply(np.array).reset_index()
client_baskets.head()

client id                       AssetClass
0          1                       [Equities]
1          2                    [Multi Asset]
2          3  [Real Estate & Private Markets]
3          4                       [Equities]
4          5                       [Equities]

### 5) Transposing grouped data into a sparse matrix
Next we transform the grouped dataset into a sparse matrix (as shown below). This is necessary to later perform apriori analytics, such as finding frequent item sets

In [10]:
te = TransactionEncoder()
te_ary = te.fit(client_baskets['AssetClass']).transform(client_baskets['AssetClass'])
dataset = pd.DataFrame(te_ary, columns=te.columns_)
dataset.head()

Distribution Partners  Equities  Fixed Income  Hedge Funds  Money Market  \
0                  False      True         False        False         False   
1                  False     False         False        False         False   
2                  False     False         False        False         False   
3                  False      True         False        False         False   
4                  False      True         False        False         False   

   Multi Asset  Real Estate & Private Markets  
0        False                          False  
1         True                          False  
2        False                           True  
3        False                          False  
4        False                          False

<b>Comment:</b> Initial look at the above shows that many clients only have invested in one asset class. This should be taken as a warning for later, because we base on out analysis on making recommendations on similar asset class "baskets". By having clients only invest in only 1 asset class we wouldn't be able to make many recommendations based on purchase patterns.

### 6) Using mlxtend's apriori method to generate a list of frequent itemsets
Below we show which item sets (asset classes) are invested in simultaneously most frequently. 

In [11]:
frequent_itemsets = apriori(dataset, min_support=0.01, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))

# if we wanted to limit itemsets to certain number we would use the following sample code: 
# frequent_itemsets = frequent_itemsets[ (frequent_itemsets['length'] >= 2)]

frequent_itemsets.sort_values('support', ascending=False)

support                                   itemsets  length
0  0.397172                                 (Equities)       1
1  0.276350                             (Fixed Income)       1
3  0.190231                              (Multi Asset)       1
4  0.156812            (Real Estate & Private Markets)       1
2  0.044987                              (Hedge Funds)       1
5  0.032134                   (Fixed Income, Equities)       2
6  0.017995                    (Equities, Multi Asset)       2
8  0.014139                (Fixed Income, Multi Asset)       2
7  0.012853  (Equities, Real Estate & Private Markets)       2

<b>Comment:</b> I choose min_support as 0.01 because we only want to focus on most frequently invested baskets. As you can see above, client accounts have mostly invested in a single asset. Having min_support as 0.01 will also let us consider most frequent baskets with 2 asset classes, however, the occurrance of this is already much lower than being invested only in a single asset class.

### 7) Using mlxtend's association_rules method to generate rules
Now we conduct the association rules analysis to be able to tell which asset classes is the client likely to be invested in, given that a client is invested in x asset class. 

I choose confidence as my metric of choice, because this will/would help us to make recommendations to other clients with similar existing investments. 

We set the confidence threshold to 1% in order to see some number of results. As described earlier, most client accounts are invested only in 1 asset, so the most likely other asset class investment would be "none".


In [12]:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.01)
rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))
rules["consequent_len"] = rules["consequents"].apply(lambda x: len(x))
rules.sort_values('confidence', ascending = False)

# if we wanted to see relationships between specific antecedent or consequent lengths 
# we sould use the following code sample: rules[ (rules['antecedent_len'] == 1) &
#                                                (rules['consequent_len'] == 2) ].sort_values('confidence', ascending = False)

antecedents                      consequents  \
0                   (Fixed Income)                       (Equities)   
3                    (Multi Asset)                       (Equities)   
5  (Real Estate & Private Markets)                       (Equities)   
1                       (Equities)                   (Fixed Income)   
7                    (Multi Asset)                   (Fixed Income)   
6                   (Fixed Income)                    (Multi Asset)   
2                       (Equities)                    (Multi Asset)   
4                       (Equities)  (Real Estate & Private Markets)   

   antecedent support  consequent support   support  confidence      lift  \
0            0.276350            0.397172  0.032134    0.116279  0.292767   
3            0.190231            0.397172  0.017995    0.094595  0.238170   
5            0.156812            0.397172  0.012853    0.081967  0.206377   
1            0.397172            0.276350  0.032134    0.080906  0.292767   
7            0.190231            0.276350  0.014139    0.074324  0.268950   
6            0.276350            0.190231  0.014139    0.051163  0.268950   
2            0.397172            0.190231  0.017995    0.045307  0.238170   
4            0.397172            0.156812  0.012853    0.032362  0.206377   

   leverage  conviction  antecedent_len  consequent_len  
0 -0.077625    0.682147               1               1  
3 -0.057560    0.665810               1               1  
5 -0.049428    0.656652               1               1  
1 -0.077625    0.787352               1               1  
7 -0.038432    0.781754               1               1  
6 -0.038432    0.853433               1               1  
2 -0.057560    0.848198               1               1  
4 -0.049428    0.871388               1               1

### 8) Analysis: 
<b>1) Individual asset class support #1:</b> client ids are most frequently exposed to Equities asset class. 39.7% of all client ids are exposed to Equities asset class

<b>2) Individual asset class support #2: </b>The most frequent combination of different asset classes for client ids is Fixed Income and Equities. 3.2% of all client ids are exposed to Fixed Income AND Equities asset classes

<b>3) Antecedent-consequent confidence #1:</b> 11.6% of client ids who are exposed to Fixed Income are also exposed to Equities. This is the highest such confidence metric that could be potentially used in fund recommentations. 

<b>4) Antecedent-consequent confidence #2:</b> The 2nd highest confidence metric is "when Multi Asset --> (then) Equities" exposure, which occurs at 9.5% of client ids

<b>5) Lift #1:</b> When looking at lift however, we notice that all shown metrics are below 1. This means that the presence of antecedent has a negative effect on presence of consequent. As such, we shouldn't use any of these rules for reliable predictions. This is caused by a large marjority of client ids being exposed to only 1 asset class (meaning client id having no other exposure is the most likely situation)
